Introduction/Business Problem: Montreal is a city covering 431.5 km^2 (166.6 mi^2) with approx. 1.8M inhabitants residing in 21 boroughs.

The City of Saints is visited by over 2M people each year; making it Canada's most visited city. With so much to see and do, I'm going to need a cup of coffee!

With that in mind, I'm going to examine The City of 100 Steeples and see which neighborhoods will stimulate weary travelers.